In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [56]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [33]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [34]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-19,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-19,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-19,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-19,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-19,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [35]:
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-19,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-19,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-19,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-19,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-19,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-19,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-19,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-19,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-19,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


# Split the Data into Training and Testing

In [36]:
# Create our features
#X = # YOUR CODE HERE
#X = df.YearsExperience.values.reshape(-1, 1)

# Define the features set.
#X = df_loans.copy()
#X = X.drop("bad", axis=1)


X = df[columns]
X = pd.get_dummies(X)


# Create our target
#y = # YOUR CODE HERE

y = df[target]

In [37]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-19,next_pymnt_d_May-19,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.994958,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.070831,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [38]:
# Check the balance of our target values
# y['loan_status'].value_counts()

y[target[0]].value_counts()


low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [39]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE

#1752

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

(51612, 97)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [40]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE# Initial imports.


from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler

#from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

#1783
# Create a random forest classifier.
rf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=78)
rf_model

BalancedRandomForestClassifier(n_estimators=128, random_state=78)

In [41]:
#1782
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[-1.23536216, -1.196855  , -1.24170452, ...,  2.46928324,
         0.        ,  0.        ],
       [ 0.22858991,  0.06540381, -0.16370179, ...,  2.46928324,
         0.        ,  0.        ],
       [-0.45458772,  0.35493183, -0.68932999, ..., -0.40497582,
         0.        ,  0.        ],
       ...,
       [-1.13776535,  0.35493183, -1.07283345, ..., -0.40497582,
         0.        ,  0.        ],
       [ 1.30215476, -0.95315156,  0.45599132, ..., -0.40497582,
         0.        ,  0.        ],
       [-0.16179731,  0.46949327,  0.13942335, ..., -0.40497582,
         0.        ,  0.        ]])

In [42]:
# Create a random forest classifier.
rf_classifier = BalancedRandomForestClassifier(n_estimators=128, random_state=78) 
rf_classifier

BalancedRandomForestClassifier(n_estimators=128, random_state=78)

In [43]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
rf_model

BalancedRandomForestClassifier(n_estimators=128, random_state=78)

In [44]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [45]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,87,0
Actual 1,14,17104


In [46]:

#balance accuracy score  fr sk learn

# Calculating the accuracy score.
#1783
acc_score = accuracy_score(y_test, predictions)

In [52]:
acc_score

0.9991862830572508

In [55]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,87,0
Actual 1,14,17104


Accuracy Score : 0.9991862830572508
Classification Report


NameError: name 'classification_report' is not defined

In [57]:
#****
#1752 Confusion Matrix
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,87,0
Actual 1,14,17104


Accuracy Score : 0.9991862830572508
Classification Report


NameError: name 'classification_report' is not defined

In [54]:
#******
# Calculated the balanced accuracy score
# YOUR CODE HERE

#1783
Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)


# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)




SyntaxError: invalid syntax (<ipython-input-54-a43a0d105f78>, line 6)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df


In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [1]:
from imblearn.metrics import classification_report_imbalanced

In [2]:
print(classification_report_imbalanced(y_test, y_pred_bal))

NameError: name 'y_test' is not defined

In [3]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE


# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


NameError: name 'rf_model' is not defined

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
#sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

### Easy Ensemble AdaBoost Classifier

In [59]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE

#1792

from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_train_scaled, y_train.ravel)
    
####
print("Learning rate: ", learning_rate)
print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train_scaled,
           y_train)))
print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test_scaled,
           y_test)))

###
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.5, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)



AttributeError: 'DataFrame' object has no attribute 'ravel'

In [ ]:
##***
# Calculated the balanced accuracy score
# YOUR CODE HERE

classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.5, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)


###1793

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

In [ ]:
#***
# Display the confusion matrix
# YOUR CODE HERE

##1793
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

In [60]:
##**# Print the imbalanced classification report
# YOUR CODE HERE

print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report


NameError: name 'classification_report' is not defined

In [ ]:
#Gradient Boosting Ada Boost

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1)

In [62]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_train_scaled, y_train)

In [66]:
print("Learning rate: ", learning_rate)
print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train_scaled,
           y_train)))
print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test_scaled,
           y_test)))

Learning rate:  1
Accuracy score (training): 0.999
Accuracy score (validation): 0.999


In [67]:
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.5, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)

In [68]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.999709386806161


In [69]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,98,3
Actual 1,2,17102


In [70]:
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.98      0.97      0.98       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.99      0.99      0.99     17205
weighted avg       1.00      1.00      1.00     17205



In [ ]:
#Random Forest Classifier

In [72]:
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [74]:
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [75]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [76]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [77]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [78]:
predictions

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [79]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,92,0
Actual 1,0,17113


In [80]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [81]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,92,0
Actual 1,0,17113


Accuracy Score : 1.0
Classification Report
              precision    recall  f1-score   support

   high_risk       1.00      1.00      1.00        92
    low_risk       1.00      1.00      1.00     17113

    accuracy                           1.00     17205
   macro avg       1.00      1.00      1.00     17205
weighted avg       1.00      1.00      1.00     17205



In [82]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,92,0
Actual 1,0,17113


Accuracy Score : 1.0
Classification Report
              precision    recall  f1-score   support

   high_risk       1.00      1.00      1.00        92
    low_risk       1.00      1.00      1.00     17113

    accuracy                           1.00     17205
   macro avg       1.00      1.00      1.00     17205
weighted avg       1.00      1.00      1.00     17205



In [83]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([2.03004646e-03, 1.93928909e-03, 1.90839830e-03, 1.85627071e-03,
       2.07153513e-03, 9.89017389e-05, 7.05426701e-04, 1.53578823e-03,
       8.72881203e-05, 1.98285051e-03, 1.50765766e-03, 2.81259801e-03,
       1.99267300e-03, 4.12661560e-02, 3.45652058e-02, 2.68532727e-02,
       4.97299854e-02, 6.00633920e-03, 0.00000000e+00, 0.00000000e+00,
       4.32386464e-02, 1.67603577e-04, 0.00000000e+00, 0.00000000e+00,
       8.35909599e-04, 1.41173057e-03, 6.86072926e-04, 1.03800548e-03,
       5.62218192e-04, 5.85717383e-04, 1.05632577e-03, 1.52286569e-03,
       1.06769804e-03, 7.33152157e-04, 1.14068449e-03, 2.48918406e-03,
       1.57614493e-03, 1.69448519e-03, 9.27319068e-04, 1.45246162e-03,
       1.54741064e-03, 1.15545176e-03, 1.41512441e-03, 2.16340751e-03,
       1.94621897e-03, 7.34167831e-05, 0.00000000e+00, 2.18677426e-03,
       2.39475692e-03, 1.79964924e-03, 1.62508892e-03, 7.88784673e-04,
       1.27221717e-03, 2.00217006e-03, 7.37703807e-04, 1.06739949e-03,
      

In [84]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3660364792760959, 'loan_status_high_risk'),
 (0.34190232782009744, 'loan_status_low_risk'),
 (0.049729985417628256, 'total_rec_int'),
 (0.043238646364785145, 'last_pymnt_amnt'),
 (0.04126615603131537, 'total_pymnt'),
 (0.034565205757814084, 'total_pymnt_inv'),
 (0.026853272678572168, 'total_rec_prncp'),
 (0.006006339203356831, 'total_rec_late_fee'),
 (0.0028125980054215585, 'out_prncp'),
 (0.0024891840594223435, 'max_bal_bc'),
 (0.0023947569230690444, 'mo_sin_old_rev_tl_op'),
 (0.0021867742567292857, 'mo_sin_old_il_acct'),
 (0.002163407510136912, 'bc_open_to_buy'),
 (0.0021619456119565173, 'num_actv_rev_tl'),
 (0.002090172334433379, 'issue_d_Mar-19'),
 (0.002078645075562943, 'num_rev_tl_bal_gt_0'),
 (0.0020715351258291373, 'dti'),
 (0.0020360074993787847, 'total_bc_limit'),
 (0.0020300464617834537, 'loan_amnt'),
 (0.0020230574013466057, 'total_bal_ex_mort'),
 (0.0020021700596835413, 'mths_since_recent_inq'),
 (0.001992672996241509, 'out_prncp_inv'),
 (0.001982850508675867, 'revol_b